In [1]:
import os
import datetime 
import sys
import xlsxwriter
import pandas as pd
from pandas import DataFrame
from openpyxl.reader.excel import load_workbook
import numpy as np
import cgi
from sklearn.cluster import KMeans 
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

class WorkStatus:
    def __init__(self,userid,satisfaction_value):
        
        self.Emp_attrition_path=os.path.join(os.path.join(os.getcwd(),"EmployeeAttrition.xlsx"))
        if os.path.isfile(self.Emp_attrition_path):
            try:
                self.worksheet0=pd.read_excel(self.Emp_attrition_path, sheet_name=0)
                self.worksheet1=pd.read_excel(self.Emp_attrition_path, sheet_name=1)
                
            except:
                print("All two sheet missing!")
                sys.exit()
        else:
            print("EmployeeAttrition.xlsx not found!")
            sys.exit()
        self.userid=userid
        self.level=str(self.level_types(int(satisfaction_value)))
        self.week_number=datetime.datetime.utcnow().isocalendar()[1]
        self.emp_predict_workSatisfy_col=5
        self.emp_predict_workValue_col=6
        
    def reopen_user_excel(self):
        if os.path.isfile(self.Emp_attrition_path):
            try:
                self.worksheet0=pd.read_excel(self.Emp_attrition_path, sheet_name=0)
                self.worksheet1=pd.read_excel(self.Emp_attrition_path, sheet_name=1)
            except:
                print("All two sheet missing!")
                sys.exit()
        else:
            print("EmployeeAttrition.xlsx not found!")
            sys.exit()
            
    def level_types(self,rating):
        if (rating > 85):           #Challenging
            return 6
        elif(rating > 70):          #Too Hard
            return 5
        elif(rating > 35):          #Hard
            return 4
        elif(rating == 35):         #Normal
            return 3
        elif(rating > 10):          #Easy
            return 2
        else:                       #Very Easy
            return 1
        
    def work_value_type(self,recognition):
        if (recognition=="reward"):
            return 0.25
        elif (recognition=="increment"):
            return 1
        elif (recognition=="promotion"):
            return 1
        elif (recognition=="onsite"):
            return 0.5
        else:
            return 0.25
        
    def satisfaction_formula(self,ticket_values):
        
        single_value=ticket_values.split(",")
        square_sum=0
        total_record=len(single_value)
        total_ticket=0
        
        for i in range(total_record):
            total_ticket+=int(single_value[i][1:])
            
        for i in range(total_record-1):            
            square_sum+=np.square(int(single_value[i+1][0])-int(single_value[i][0]))+np.square(int(single_value[i+1][1:])-int(single_value[i][1:]))
       
        return np.around(np.sqrt(square_sum/(total_record*total_ticket)),decimals=4)
           
    def search_user(self,sheet):
        for rowno in range(sheet.shape[0]):
            if (str(sheet.iloc[rowno,0]) == self.userid):
                return rowno
        
        return None
        
    def read_user_cell(self,UserNum,week_value): 
        return self.worksheet0.iloc[UserNum,week_value]
    
    def read_user_work_value(self,UserNum):
        return self.worksheet1.iloc[UserNum,self.emp_predict_workValue_col]
    
    def save_excel(self,sheetno,row_number,column_number,data):
        wb = load_workbook(filename=self.Emp_attrition_path)
        ws = wb.worksheets[sheetno]
        ws.cell(row=row_number+2, column=column_number+1).value = data
        wb.save(self.Emp_attrition_path) 
        
    def write_update_user(self):
        rowno=self.search_user(self.worksheet0)
        if(rowno!=None):
            user_levels=str(self.read_user_cell(rowno,self.week_number))
            
            if (user_levels=="nan"):
                self.save_excel(0,rowno,self.week_number,int(str(self.level)+"1"))
            else :  
                if (str(self.level)==user_levels.split(",")[-1][0]):
                    if("," in user_levels):
                        self.save_excel(0,rowno,self.week_number,str(','.join(user_levels.split(",")[:-1])+","+str(self.level)+str(int(user_levels.split(",")[-1][1:])+1)))
                    else:
                        self.save_excel(0,rowno,self.week_number,str(str(self.level)+str(int(user_levels.split(".")[0][1:])+1)))
                else:
                    if("," in user_levels):
                        self.save_excel(0,rowno,self.week_number,str(user_levels+","+str(self.level)+"1"))
                    else:
                        self.save_excel(0,rowno,self.week_number,str(user_levels.split(".")[0]+","+str(self.level)+"1"))
            
        else:
            self.save_excel(0,int(self.worksheet0.shape[0]),self.week_number,(str(self.level)+"1"))
            self.save_excel(0,int(self.worksheet0.shape[0]),0,self.userid)
        
        self.reopen_user_excel()
        self.update_work_satisfaction()
                
    def update_work_satisfaction(self):
        
        distribution=[]
        row=self.search_user(self.worksheet0)
        row_predict=self.search_user(self.worksheet1)
        
        if(row==None or row_predict==None):
            print("User Not found in EmployeeAttrition.xlsx")
            sys.exit()
        else:
            for col in range(1,int(self.worksheet0.shape[1])):
                user_levels=self.read_user_cell(row,col)
                if(type(user_levels) is str):
                    distribution.append(self.satisfaction_formula(user_levels))
                elif(np.isfinite(user_levels)):
                    single_square_sum=np.square(int(str(user_levels)[0]))+np.square(int(str(user_levels)[1:].split(".")[0]))
                    distribution.append(np.around(np.sqrt(single_square_sum/int((str(user_levels)[1:].split(".")[0]))),decimals=4))
                    
        self.save_excel(1,row_predict,self.emp_predict_workSatisfy_col,np.around((np.std(distribution)-1),decimals=4))
        self.reopen_user_excel()
        
        
    def update_work_value(self,user_value):
        row_predict=self.search_user(self.worksheet1)
        
        if(row_predict==None):
            print("User Not found in EmployeeAttrition.xlsx")
            sys.exit()
        else:
            work_value=self.read_user_work_value(row_predict)
            if (np.isfinite(work_value)):
                self.save_excel(1,row_predict,self.emp_predict_workValue_col,work_value+self.work_value_type(user_value))
                self.reopen_user_excel()
            else:
                self.save_excel(1,row_predict,self.emp_predict_workValue_col,self.work_value_type(user_value))
                self.reopen_user_excel()
        
    def kmean_clustering(self):
        '''classes = self.worksheet1['AttritionPredict']  
        self.worksheet1 = self.worksheet1.drop(['UserId','AttritionPredict'],axis=1) 
        ## Convert dataframe into list and then into a numpy array
        data = self.worksheet1.values.tolist() 
        data = np.array(data)
        ## Shuffle classes and data 
        scores=[]
        for count in range(100):
            data,classes = shuffle(data,classes) 
            train_data = data[:10]  
            test_data = data[10:]

            attrition = KMeans(n_clusters = 2)
            attrition.fit(train_data)
            pred = attrition.predict(test_data)
            
            scores.append(metrics.adjusted_rand_score(pred,classes[10:]))
            #scores.append(accuracy_score(pred,classes[10:]))
        
        print(np.mean(scores))'''
        
        scores=[]
        for count in range(100):
            target = self.worksheet1['AttritionPredict']
            features = self.worksheet1.drop(['UserId','AttritionPredict'], axis = 1)
            #create the train/test split
            X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.4, random_state=10)
            #Create the model and train
            model = RandomForestClassifier()
            model.fit(X_train,y_train)
            #predict the results for test
            test_pred = model.predict(X_test)
            #test the accuracy
            scores.append(accuracy_score(y_test, test_pred))
        
        print(np.mean(scores))


C:\Users\agupt118\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#cgitb.enable()
#form = cgi.FieldStorage()
#userlevel = form.getvalue('satisfaction_value')
#UserStatus=WorkStatus(os.getlogin(),28)
for user in (["RAMSES",
"STANISLAV",
"EBRIMA",
"ABDALLAH",
"ALIOU",
"VINCENZO",
"YUSEF",
"HONORATO",
"IRAITZ",
"FREDDY",
"FRANCIS",
"KHALED",
"LIBERTO",
"GUIDO"]):

    UserStatus=WorkStatus(user,86)
    UserStatus.write_update_user()
    #UserStatus.update_work_satisfaction()
UserStatus.kmean_clustering()
#UserStatus.update_work_value("reward")

0.6833333333333332
